# 极客云训练深度学习模型 Assembly Line
> * 上传数据和代码
> * 运行程序
> * 下载训练好的模型

## 1.1 数据预处理后，将模型的输入数据保存为文件

In [ ]:
with open('embedding_weights.pkl','wb')as f:
    pickle.dump(embedding_weights,f)
    print('embedding_weights'+'存入文件成功')

with open('x_test.pkl','wb')as f:
    pickle.dump(x_test,f) 
    print('x_test'+'存入文件成功')
    
with open('y_test.pkl','wb')as f:
    pickle.dump(y_test,f)
    print('y_test'+'存入文件成功')

with open('y_train.pkl','wb')as f:
    pickle.dump(y_train,f)  
    print('y_train'+'存入文件成功')
    
with open('x_train.pkl','wb')as f:
    pickle.dump(x_train,f)
    print('x_train'+'存入文件成功')

## 1.2 输入数据的读入
##### 注意初始化一些必要参数

In [ ]:
#以下是 模型需要的数据一共五个大文件，加上两个参数   
with open('/input/x_test.pkl','rb')as f:
    x_test=pickle.load(f)
    print('x_test'+'载入文件成功')

with open('/input/y_test.pkl','rb')as f:
    y_test=pickle.load(f)
    print('y_test'+'载入文件成功')

    
with open('/input/x_train.pkl','rb')as f:
    x_train=pickle.load(f)
    print('x_train'+'载入文件成功')

    
with open('/input/y_train.pkl','rb')as f:
    y_train=pickle.load(f) 
    print('y_train'+'载入文件成功')

    
with open('/input/embedding_weights.pkl','rb')as f:
    embedding_weights=pickle.load(f) 
    print('embedding_weights'+'载入文件成功')

 
len_vocabulary_inv=103943
y_shape_1=202
print("Load data success...")

## 1.3 模型

In [ ]:
# ---------------------- Parameters section -------------------
#
# Model type. See Kim Yoon's Convolutional Neural Networks for Sentence Classification, Section 3
model_type = "CNN-non-static"  # CNN-rand|CNN-non-static|CNN-static
#model_type = "CNN-static"  # CNN-rand|CNN-non-static|CNN-static

# Model Hyperparameters
#embedding_dim = 100
embedding_dim = 500

filter_sizes = (3, 8)
#filter_sizes = (6, 8)
#num_filters = 128
#num_filters = 10
num_filters = 50
dropout_prob = (0.5, 0.8)
hidden_dims = 50

# Training parameters
#batch_size = 32
batch_size = 64
num_epochs = 50
#num_epochs = 1

# Prepossessing parameters
#sequence_length = 1500   #400
sequence_length = 500
#sequence_length = 400   #400

#max_words = 20000   #5000

# Word2Vec parameters (see train_word2vec)
min_word_count = 1
context = 10


In [ ]:
#==============================================================================
# # Build model
#==============================================================================
def text_cnn():
    if model_type == "CNN-static":
        input_shape = (sequence_length, embedding_dim)
    else:
        input_shape = (sequence_length,)
    
    model_input = Input(shape=input_shape)
    
    # Static model does not have embedding layer
#    嵌入层Embedding被定义为网络的第一个隐藏层。它必须指定3个参数：
#    它必须指定3个参数：
#    1.input_dim：这是文本数据中词汇的大小。 
#    2.output_dim：这是嵌入单词的向量空间的大小。它为每个单词定义了该层的输出向量的大小。例如，它可以是32或100甚至更大。根据你的问题来定。
#    3.input_length：这是输入序列的长度，正如你为Keras模型的任何输入层定义的那样。例如，如果你的所有输入文档包含1000个单词，则为1000
    if model_type == "CNN-static":
        z = model_input
    else:
        z = Embedding(len_vocabulary_inv, embedding_dim, input_length=sequence_length, name="embedding")(model_input)
    
    z = Dropout(dropout_prob[0])(z)
    
    # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,
                             kernel_size=sz,
                             padding="valid",
                             activation="relu",
                             strides=1)(z)
        conv = MaxPooling1D(pool_size=2)(conv)
        conv = Flatten()(conv)
        conv_blocks.append(conv)
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
    
    z = Dropout(dropout_prob[1])(z)
    z = Dense(hidden_dims, activation="relu")(z)
    model_output = Dense(y_shape_1, activation="softmax")(z)
    
    model = Model(model_input, model_output)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    #==============================================================================
    # # Initialize weights with word2vec
    #==============================================================================
    if model_type == "CNN-non-static":
        weights = np.array([v for v in embedding_weights.values()])
        print("Initializing embedding layer with word2vec weights, shape", weights.shape)
        embedding_layer = model.get_layer("embedding")
        embedding_layer.set_weights([weights])

    return model


## 1.4 训练及模型保存

In [ ]:
#==============================================================================
# # Train the model
#==============================================================================
print("training...")
model=text_cnn()
model.fit(x_train, y_train,validation_split=0.2, batch_size=batch_size, epochs=num_epochs, verbose=2,class_weight='auto')

print("pridicting...")
scores = model.evaluate(x_test,y_test)
print('test_loss:%f,accuracy: %f'%(scores[0],scores[1]))

print("saving accu_textcnnmodel")
model.save('/data/textcnn_keras_law/accu_textcnn.h5')

## 1.5 上传数据和代码
> * 使用Winscp，上传到/data文件夹，再把数据都复制到/input文件夹,
> * "cp空格/data/xxx/mydata.pkl空格/input"



## 1.6 日志（可选）
> * 程序打印输出的内容将不再出现在终端上，而是保存在log文件里

In [ ]:
#日志
import sys
f_handler=open('out.log', 'w')
sys.stdout=f_handler

## 最终日志结果

In [ ]:
x_test载入文件成功
y_test载入文件成功
x_train载入文件成功
y_train载入文件成功
embedding_weights载入文件成功
Load data success...
training...
Initializing embedding layer with word2vec weights, shape (103943, 500)
Train on 10963 samples, validate on 2741 samples
Epoch 1/50
10s - loss: 4.6456 - acc: 0.0506 - val_loss: 4.4022 - val_acc: 0.0905
Epoch 2/50
8s - loss: 3.9241 - acc: 0.1506 - val_loss: 3.4705 - val_acc: 0.2328
Epoch 3/50
8s - loss: 2.8834 - acc: 0.3070 - val_loss: 2.6227 - val_acc: 0.3652
Epoch 4/50
8s - loss: 2.0999 - acc: 0.4601 - val_loss: 2.1469 - val_acc: 0.4943
Epoch 5/50
8s - loss: 1.5671 - acc: 0.5739 - val_loss: 1.9125 - val_acc: 0.5509
Epoch 6/50
8s - loss: 1.2058 - acc: 0.6651 - val_loss: 1.7807 - val_acc: 0.5834
Epoch 7/50
8s - loss: 0.9553 - acc: 0.7253 - val_loss: 1.7046 - val_acc: 0.6053
Epoch 8/50
8s - loss: 0.7701 - acc: 0.7749 - val_loss: 1.7169 - val_acc: 0.6224
Epoch 9/50
8s - loss: 0.6507 - acc: 0.8130 - val_loss: 1.6808 - val_acc: 0.6425
Epoch 10/50
8s - loss: 0.5269 - acc: 0.8422 - val_loss: 1.7212 - val_acc: 0.6432
Epoch 11/50
8s - loss: 0.4769 - acc: 0.8561 - val_loss: 1.7688 - val_acc: 0.6461
Epoch 12/50
8s - loss: 0.4127 - acc: 0.8773 - val_loss: 1.7394 - val_acc: 0.6571
Epoch 13/50
8s - loss: 0.3655 - acc: 0.8864 - val_loss: 1.8485 - val_acc: 0.6592
Epoch 14/50
8s - loss: 0.3443 - acc: 0.8964 - val_loss: 1.9085 - val_acc: 0.6571
Epoch 15/50
8s - loss: 0.3084 - acc: 0.9041 - val_loss: 1.9446 - val_acc: 0.6545
Epoch 16/50
8s - loss: 0.2740 - acc: 0.9164 - val_loss: 2.0552 - val_acc: 0.6662
Epoch 17/50
8s - loss: 0.2545 - acc: 0.9217 - val_loss: 2.0268 - val_acc: 0.6603
Epoch 18/50
8s - loss: 0.2339 - acc: 0.9287 - val_loss: 2.0788 - val_acc: 0.6611
Epoch 19/50
8s - loss: 0.2265 - acc: 0.9308 - val_loss: 2.0727 - val_acc: 0.6669
Epoch 20/50
8s - loss: 0.2102 - acc: 0.9335 - val_loss: 2.1260 - val_acc: 0.6665
Epoch 21/50
8s - loss: 0.2084 - acc: 0.9403 - val_loss: 2.1841 - val_acc: 0.6651
Epoch 22/50
8s - loss: 0.2035 - acc: 0.9393 - val_loss: 2.1881 - val_acc: 0.6779
Epoch 23/50
8s - loss: 0.1997 - acc: 0.9403 - val_loss: 2.2483 - val_acc: 0.6636
Epoch 24/50
8s - loss: 0.1800 - acc: 0.9484 - val_loss: 2.2823 - val_acc: 0.6665
Epoch 25/50
8s - loss: 0.1807 - acc: 0.9476 - val_loss: 2.2828 - val_acc: 0.6698
Epoch 26/50
8s - loss: 0.1720 - acc: 0.9500 - val_loss: 2.3505 - val_acc: 0.6749
Epoch 27/50
8s - loss: 0.1748 - acc: 0.9484 - val_loss: 2.3895 - val_acc: 0.6713
Epoch 28/50
8s - loss: 0.1692 - acc: 0.9507 - val_loss: 2.3226 - val_acc: 0.6691
Epoch 29/50
8s - loss: 0.1721 - acc: 0.9515 - val_loss: 2.3920 - val_acc: 0.6731
Epoch 30/50
8s - loss: 0.1838 - acc: 0.9480 - val_loss: 2.3880 - val_acc: 0.6757
Epoch 31/50
8s - loss: 0.1858 - acc: 0.9476 - val_loss: 2.4445 - val_acc: 0.6764
Epoch 32/50
8s - loss: 0.1766 - acc: 0.9488 - val_loss: 2.4153 - val_acc: 0.6717
Epoch 33/50
8s - loss: 0.1598 - acc: 0.9550 - val_loss: 2.5468 - val_acc: 0.6585
Epoch 34/50
8s - loss: 0.1404 - acc: 0.9600 - val_loss: 2.4975 - val_acc: 0.6830
Epoch 35/50
8s - loss: 0.1458 - acc: 0.9579 - val_loss: 2.5573 - val_acc: 0.6757
Epoch 36/50
8s - loss: 0.1517 - acc: 0.9569 - val_loss: 2.5365 - val_acc: 0.6738
Epoch 37/50
8s - loss: 0.1699 - acc: 0.9524 - val_loss: 2.6479 - val_acc: 0.6789
Epoch 38/50
8s - loss: 0.1529 - acc: 0.9570 - val_loss: 2.6829 - val_acc: 0.6746
Epoch 39/50
8s - loss: 0.1404 - acc: 0.9620 - val_loss: 2.7596 - val_acc: 0.6662
Epoch 40/50
8s - loss: 0.1520 - acc: 0.9614 - val_loss: 2.6221 - val_acc: 0.6735
Epoch 41/50
8s - loss: 0.1593 - acc: 0.9554 - val_loss: 2.7454 - val_acc: 0.6713
Epoch 42/50
8s - loss: 0.1345 - acc: 0.9637 - val_loss: 2.7122 - val_acc: 0.6844
Epoch 43/50
8s - loss: 0.1336 - acc: 0.9666 - val_loss: 2.6978 - val_acc: 0.6808
Epoch 44/50
8s - loss: 0.1481 - acc: 0.9615 - val_loss: 2.6845 - val_acc: 0.6873
Epoch 45/50
8s - loss: 0.1450 - acc: 0.9642 - val_loss: 2.7867 - val_acc: 0.6753
Epoch 46/50
8s - loss: 0.1292 - acc: 0.9665 - val_loss: 2.7196 - val_acc: 0.6815
Epoch 47/50
8s - loss: 0.1244 - acc: 0.9670 - val_loss: 2.8670 - val_acc: 0.6782
Epoch 48/50
8s - loss: 0.1601 - acc: 0.9623 - val_loss: 2.8403 - val_acc: 0.6771
Epoch 49/50
8s - loss: 0.1532 - acc: 0.9619 - val_loss: 2.7765 - val_acc: 0.6859
Epoch 50/50
8s - loss: 0.1333 - acc: 0.9648 - val_loss: 2.8925 - val_acc: 0.6706
pridicting...

  32/3427 [..............................] - ETA: 1s
 384/3427 [==>...........................] - ETA: 0s
 736/3427 [=====>........................] - ETA: 0s
1088/3427 [========>.....................] - ETA: 0s
1440/3427 [===========>..................] - ETA: 0s
1792/3427 [==============>...............] - ETA: 0s
2144/3427 [=================>............] - ETA: 0s
2464/3427 [====================>.........] - ETA: 0s
2816/3427 [=======================>......] - ETA: 0s
3168/3427 [==========================>...] - ETA: 0stest_loss:2.805001,accuracy: 0.681938
saving accu_textcnnmodel
